## Empirical results

In [3]:
import os
import sys
import shutil
import pandas
import numpy
import json

import ipystata
from ipystata.config import config_stata
config_stata('/usr/local/stata/stata-mp')

pandas.set_option('display.float_format', lambda x: '%.4f' % x)
pandas.set_option('max_columns', 200)
pandas.set_option('max_rows', 400)
pandas.set_option('max_colwidth', 150)
pandas.set_option('mode.sim_interactive', True)
pandas.set_option('colheader_justify', 'center')
pandas.set_option('precision', 4)

from IPython.display import display, display_markdown, Markdown, HTML, IFrame
# from IPython.display import display, HTML, Image, IFrame, publish_display_data, display_markdown
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# pandas.options.display.float_format = '{:.2f}'.format

In [4]:
%cd /mnt/intel1200/projects/git/exercise_physiology/doc

/mnt/intel1200/projects/git/exercise_physiology/doc


In [5]:
dm = pandas.read_csv('data_analysis.csv')

de = dm.copy()
de = de[['id', 'trip', 'st_rsos', 'st_tsos', 'st_grip', 'st_biodex', 'st_ntxc', 'st_matu', 'st_calo', 'st_mvh', 'gender']]
de = de.set_index(['id', 'trip'])

de['boy']  = numpy.where(de.gender=="boy", 1, 0)
de['girl'] = numpy.where(de.gender=="girl", 1, 0)

de = de.rename(columns={'st_rsos': 'r'})
de = de.rename(columns={'st_tsos': 't'})
de = de.rename(columns={'st_grip': 'g'})
de = de.rename(columns={'st_biodex': 'b'})
de = de.rename(columns={'st_ntxc': 'n'})
de = de.rename(columns={'st_matu': 'm'})
de = de.rename(columns={'st_calo': 'c'})
de = de.rename(columns={'st_mvh': 'v'})
de = de.rename(columns={'gender': 'G'})

de.to_csv('de.csv')
de.head(2)

r       t      g       b      n       m       c       v    G    \
id  trip                                                                     
100 1    0.1192 -0.7897    nan -0.0412 0.6525 -0.4026 -0.0447 -0.0322  boy   
    2    0.8191 -0.5338 0.4830  0.1251 0.8355  0.0729  1.3046 -0.3233  boy   

          boy  girl  
id  trip             
100 1      1     0   
    2      1     0

In [ ]:
%%stata --data de 
display c(current_time) 

gen BOYRadi = boy*r
gen BOYTibi = boy*t
gen BOYGrip = boy*g
gen BOYBiod = boy*b

gen GIRRadi = girl*r
gen GIRTibi = girl*t
gen GIRGrip = girl*g
gen GIRBiod = girl*b


global R1 "r <- g@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global T1 "t <- b@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global M1  "g b <- m@c4 c@k5 P2[id]@1 _cons@km"

global R2 "r <- g m n@k3 P1[id]@1 _cons@kk"
global T2 "t <- b m n@k3 P1[id]@1 _cons@kk"
global M2  "g b <- m c@k5 P2[id]@1 _cons@km"

global L "covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2)"
global C "cov(e.r*e.t) cov(e.g*e.b) cov(P1[id]*P2[id])"
global O "nohead nolog"


quietly gsem ///
(BOYRadi BOYTibi <- m n) ///
(BOYRadi <- BOYGrip) ///
(BOYTibi <- BOYBiod) ///
(BOYGrip BOYBiod <- m c) ///
///
(GIRRadi GIRTibi <- m n ) ///
(GIRRadi <- GIRGrip) ///
(GIRTibi <- GIRBiod) ///
(GIRGrip GIRBiod <- m c) ///
, nocapslatent   ///
covstruct(_lexogenous, diagonal) ///
cov(e.BOYRadi*e.BOYTibi) cov(e.BOYGrip*e.BOYBiod)  ///
cov(e.GIRRadi*e.GIRTibi) cov(e.GIRGrip*e.GIRBiod)  ///
nohead nolog 

matrix b = e(b)

quietly gsem ///
(BOYRadi BOYTibi <- m n@BK1  P1[id]@1 _cons@BK3) ///
(BOYRadi <- BOYGrip) ///
(BOYTibi <- BOYBiod) ///
(BOYGrip BOYBiod <- m c@BK2 P2[id]@1 _cons@BK4) ///
///
(GIRRadi GIRTibi <- m n@GK1 G1[id]@1  _cons@GK3) ///
(GIRRadi <- GIRGrip) ///
(GIRTibi <- GIRBiod) ///
(GIRGrip GIRBiod <- m c@GK2  G2[id]@1 _cons@GK4) ///
, nocapslatent latent(P1 P2 G1 G2)  ///
covstruct(_lexogenous, diagonal) ///
cov(e.BOYRadi*e.BOYTibi) cov(e.BOYGrip*e.BOYBiod) ///
cov(e.GIRRadi*e.GIRTibi) cov(e.GIRGrip*e.GIRBiod)  ///
cov(P1[id]*G1[id]@0) cov(P1[id]*G2[id]@0) cov(P2[id]*G1[id]@0) cov(P2[id]*G2[id]@0) ///
nohead nolog from(b, skip)
estimates save BOTH, replace
estimates store BOTH

esttab BOTH

In [ ]:
%%stata --data de 
display c(current_time) 

gen BOYRadi = boy*r
gen BOYTibi = boy*t
gen BOYGrip = boy*g
gen BOYBiod = boy*b

gen GIRRadi = girl*r
gen GIRTibi = girl*t
gen GIRGrip = girl*g
gen GIRBiod = girl*b


encode(G), gen(GX)

label variable r "Radial SOS"
label variable r "Tibial SOS"
label variable g "Grip strength"
label variable b "Knee extensor"
label variable m "Physical maturation"
label variable n "Ntx creatine"
label variable b "Knee extensor"
label variable c "Energy intake"

label variable BOYRadi "Radial SOS"
label variable BOYTibi "Tibial SOS"
label variable GIRRadi "Radial SOS"
label variable GIRTibi "Tibial SOS"

label variable BOYGrip "Grip strength"
label variable BOYBiod "Knee extensor"
label variable GIRGrip "Grip strength"
label variable GIRBiod "Knee extensor"

display "Boys and girls model: combined specificaition"
display "*********************************************"


global R1 "r <- g@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global T1 "t <- b@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global M1  "g b <- m@c4 c@k5 P2[id]@1 _cons@km"

global L "covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2)"
global C "cov(e.r*e.t) cov(e.g*e.b) cov(P1[id]*P2[id])"
global O "nohead nolog"


gsem ///
    (r <- g@k1 m@k2 n@k3 _cons@kk) ///
    (t <- b@k1 m@k2 n@k3 _cons@kk), ///
    covstruct(_lexogenous, diagonal) cov(e.r*e.t)

gsem ///
    (r <- m@k2 n@k3 _cons@kk) ///
    (t <- m@k2 n@k3 _cons@kk) ///
    , covstruct(_lexogenous, diagonal) cov(e.r*e.t)
    

In [ ]:
%%stata --data de 
display c(current_time) 

gen BOYRadi = boy*r
gen BOYTibi = boy*t
gen BOYGrip = boy*g
gen BOYBiod = boy*b

gen GIRRadi = girl*r
gen GIRTibi = girl*t
gen GIRGrip = girl*g
gen GIRBiod = girl*b


encode(G), gen(GX)

label variable r "Radial SOS"
label variable r "Tibial SOS"
label variable g "Grip strength"
label variable b "Knee extensor"
label variable m "Physical maturation"
label variable n "Ntx creatine"
label variable b "Knee extensor"
label variable c "Energy intake"

label variable BOYRadi "Radial SOS"
label variable BOYTibi "Tibial SOS"
label variable GIRRadi "Radial SOS"
label variable GIRTibi "Tibial SOS"

label variable BOYGrip "Grip strength"
label variable BOYBiod "Knee extensor"
label variable GIRGrip "Grip strength"
label variable GIRBiod "Knee extensor"

display "Boys and girls model: combined specificaition"
display "*********************************************"


global R1 "r <- g@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global T1 "t <- b@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global M1  "g b <- m@c4 c@k5 P2[id]@1 _cons@km"

global L "covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2)"
global C "cov(e.r*e.t) cov(e.g*e.b) cov(P1[id]*P2[id])"
global O "nohead nolog"




gsem (r <- g@k1 m@k2 n@k3 _cons@kk) ///
(t <- b@k1 m@k2 n@k3  _cons@kk) ///
(g b <- m@c4 c@k5  _cons@km) ///
, covstruct(_lexogenous, diagonal) nohead nolog
estimates store M1

 gsem ///
(r <- g@k1 m@k2 n@k3 c@k4 _cons@kk) ///
(t <- b@k1 m@k2 n@k3 c@k4 _cons@kk) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2) ///
nohead nolog

estimates store N1

esttab M1 N1
    

In [ ]:
%%stata --data de 
display c(current_time) 

gen BOYRadi = boy*r
gen BOYTibi = boy*t
gen BOYGrip = boy*g
gen BOYBiod = boy*b

gen GIRRadi = girl*r
gen GIRTibi = girl*t
gen GIRGrip = girl*g
gen GIRBiod = girl*b


encode(G), gen(GX)

label variable r "Radial SOS"
label variable t "Tibial SOS"
label variable g "Grip strength"
label variable b "Knee extensor"
label variable m "Physical maturation"
label variable n "Ntx creatinine"
label variable b "Knee extensor"
label variable c "Energy intake"

label variable BOYRadi "Radial SOS"
label variable BOYTibi "Tibial SOS"
label variable GIRRadi "Radial SOS"
label variable GIRTibi "Tibial SOS"

label variable BOYGrip "Grip strength"
label variable BOYBiod "Knee extensor"
label variable GIRGrip "Grip strength"
label variable GIRBiod "Knee extensor"

display "Boys and girls model: combined specificaition"
display "*********************************************"


global R1 "r <- g@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global T1 "t <- b@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global M1  "g b <- m@c4 c@k5 P2[id]@1 _cons@km"

global R2 "r <- g m n@k3 P1[id]@1 _cons@kk"
global T2 "t <- b m n@k3 P1[id]@1 _cons@kk"
global M2  "g b <- m c@k5 P2[id]@1 _cons@km"

global R3 "r <- g m n P1[id]"
global T3 "t <- b m n P1[id]"
global M3  "g b <- m c P2[id]"

global L "covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2)"
global C "cov(e.r*e.t) cov(e.g*e.b) cov(P1[id]*P2[id])"
global O "nohead nolog"

quietly gsem ($R1) ($T1) ($M1), $C $L $O
estimates save M1, replace
estimates store M1
matrix b = e(b)

quietly gsem ($R3) ($T3) ($M3), $C $L $O from(b, skip)
estimates save M2, replace
estimates store M2

/*

quietly gsem ($R2) ($T2) ($M2), $C $L $O
estimates save M2, replace
estimates store M2
matrix b = e(b)

quietly gsem ($R3) ($T3) ($M3), $C $L $O from(b, skip)
estimates save M3, replace
estimates store M3

quietly gsem ($R1) ($T1) ($M1) if boy==1, $C $L $O
estimates save M4, replace
estimates store M4

quietly gsem ($R1) ($T1) ($M1) if boy==0, $C $L $O
estimates save M5, replace
estimates store M5
*/

esttab M1 M2 , ///
label b(%9.4f) se(%9.4f)  star(* 0.10 ** 0.05 *** 0.01) ///
nodepvars nonumbers parentheses noeqlines compress nonotes ///
mtitles("General" "Specific" "Boys") ///
title("Table 2. Estiamtion results") ///
addnotes("Notes: Standard errors in parentheses. All level-1 fixed effects and level-2 random effects were found to be significant at the 0.05 level") ///
coeflabels(var(P1[id]):_cons "Bone equations" var(P2[id]):_cons "Muscle equations" cov(P1[id],P2[id]):_cons "Covariance") ///
varwidth(20) modelwidth(15)

In [ ]:
%%stata 


estimates use M1
estimates store M1

estimates use M2
estimates store M2

estimates stats M1 M2

In [ ]:
# %load models.do
*cd "/mnt/intel1200/projects/git/exercise_physiology/out/"

*import delimited /mnt/intel1200/projects/git/exercise_physiology/out/de.csv, clear case(preserve)

global R "r <- g@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global T "t <- b@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global M "g b <- m@k4 c@k5 M2[id]@1 _cons@km"
global C "covstruct(_lexogenous, diagonal) nocapslatent latent(M1 M2) cov(e.r*e.t) cov(e.g*e.b) cov(M1[id]*M2[id]) "
global O "nohead nolog"

global RS "tstat pval ci level(95)"

quietly gsem ($R) ($T) ($M), $C $O
estimates save ALL, replace
estimates store ALL

*regsave using table2, replace addlabel(M, all) tstat pval ci level(95)

quietly gsem ($R) ($T) ($M) if G == "boy", $C $O
estimates save BOYS, replace
estimates store BOYS
*regsave using table2, append addlabel(M, boys) tstat pval ci level(95)

quietly gsem ($R) ($T) ($M) if G == "girl", $C $O
estimates save GIRLS, replace
estimates store GIRLS
*regsave using table2, append addlabel(M, girls) tstat pval ci level(95)

label variable r "Radial SOS"
label variable r "Tibial SOS"
label variable g "Grip strength"
label variable b "Knee extensor"
label variable m "Physical maturation"
label variable n "Ntx creatine"
label variable b "Knee extensor"
label variable c "Energy intake"

label variable g "Muscle strength"
label variable r "Bone properties"

esttab ALL BOYS GIRLS, ///
keep(r:g r:m r:n r:_cons g:m g:c g:_cons var(M1[id]):_cons var(M2[id]):_cons cov(M2[id],M1[id]):_cons ) ///
label b(%9.4f) se(%9.4f)  nostar ///
nodepvars nonumbers parentheses noeqlines compress nonotes ///
mtitles("All" "Boys" "Girls") ///
title("Table 2. Estiamtion results") ///
addnotes("Notes: Standard errors in parentheses. All level-1 fixed effects and level-2 random effects were found to be significant at the 0.05 level") ///
coeflabels(var(M1[id]):_cons "Bone equations" var(M2[id]):_cons "Muscle equations" cov(M2[id],M1[id]):_cons "Covariance") ///
varwidth(20) modelwidth(15)


In [ ]:
%%stata --data de 

do models


In [ ]:
%%stata --data de


global R "r <- g@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global T "t <- b@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global M "g b <- m@k4 c@k5 M2[id]@1 _cons@km"
global C "covstruct(_lexogenous, diagonal) nocapslatent latent(M1 M2) cov(e.r*e.t) cov(e.g*e.b) cov(M1[id]*M2[id]) "
global O "nohead nolog"

global RS "tstat pval ci level(95)"

*gsem ($R) ($T) ($M), $C $O
*estimates store ALL

gsem ///
(r <- m n M1[id] _cons) ///
(t <- m n M2[id] _cons) ///
, covstruct(_lexogenous, diagonal) nocapslatent latent(M1 M2 ) cov(e.r*e.t)

estimates store DUMMY



In [ ]:
%%stata --data de 

gen BOY = 1 if G =="boy"
gen GIR = 1 if G =="girl"

gen BOYRadi = BOY*r
gen BOYTibi = BOY*t
gen BOYGrip = BOY*g
gen BOYBiod = BOY*b

gen GIRRadi = GIR*r
gen GIRTibi = GIR*t
gen GIRGrip = GIR*g
gen GIRBiod = GIR*b

label variable r "Radial SOS"
label variable r "Tibial SOS"
label variable g "Grip strength"
label variable b "Knee extensor"
label variable m "Physical maturation"
label variable n "Ntx creatine"
label variable b "Knee extensor"
label variable c "Energy intake"

label variable BOYRadi "Radial SOS"
label variable BOYTibi "Tibial SOS"
label variable BOYGrip "Grip strength"
label variable BOYBiod "Knee extensor"

label variable GIRRadi "Radial SOS"
label variable GIRTibi "Tibial SOS"
label variable GIRGrip "Grip strength"
label variable GIRBiod "Knee extensor"

display "Boys and girls model: combined specificaition"
display "*********************************************"


gsem ///
(BOYRadi BOYTibi <- m@BRm n@BRn B1[id]@1 _cons@BRc1) ///
(BOYRadi <- BOYGrip@BRs ) ///
(BOYTibi <- BOYBiod@BRs) ///
(BOYGrip BOYBiod <- m@BRp c@BRe B2[id]@1 _cons@BRc2) ///
///
(GIRRadi GIRTibi <- m@GRm n@GRn G1[id]@1 _cons@GRc1) ///
(GIRRadi <- GIRGrip@GRs ) ///
(GIRTibi <- GIRBiod@GRs) ///
(GIRGrip GIRBiod <- m@GRp c@GRe G2[id]@1 _cons@GRc2) ///
, nocapslatent latent(B1 B2 G1 G2)  ///
covstruct(_lexogenous, diagonal) ///
cov(e.BOYRadi*e.BOYTibi) cov(e.BOYGrip*e.BOYBiod) cov(B1[id]*B2[id]) ///
cov(e.GIRRadi*e.GIRTibi) cov(e.GIRGrip*e.GIRBiod) cov(G1[id]*G2[id]) ///
cov(B1[id]*G1[id]@0) cov(B2[id]*G2[id]@0) cov(B1[id]*G2[id]@0) ///
nohead nolog 
matrix b = e(b)
estimates save BOTH


global R1 "r <- g@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global T1 "t <- b@k1 m@k2 n@k3 P1[id]@1 _cons@kk"
global M1  "g b <- m@c4 c@k5 P2[id]@1 _cons@km"

global R2 "r <- g m n@k3 P1[id]@1 _cons@kk"
global T2 "t <- b m n@k3 P1[id]@1 _cons@kk"
global M2  "g b <- m c@k5 P2[id]@1 _cons@km"

global L "covstruct(_lexogenous, diagonal) nocapslatent latent(P1 P2)"
global C "cov(e.r*e.t) cov(e.g*e.b) cov(P1[id]*P2[id])"
global O "nohead nolog"

quietly gsem ($R1) ($T1) ($M1) if BOY==1, $C $L $O
estimates store BOY

quietly gsem ($R1) ($T1) ($M1) if BOY==0, $C $L $O
estimates store GIRL

esttab BOTH BOY GIRL



In [ ]:
%%stata 
*ssc install lincomest

estimates use BOTH
*estimate replay
esttab , label 

lincomest [BOYRadi]m - [GIRRadi]m
return list
matrix list r(table)
matrix r = get(_b)
matrix list r


In [ ]:
%%stata --data de --out dr

global R "r <- g@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global T "t <- b@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global M "g b <- m@k4 c@k5 M2[id]@1"
global C "covstruct(_lexogenous, diagonal) nocapslatent latent(M1 M2) cov(e.r*e.t) cov(e.g*e.b) cov(M1[id]*M2[id]) "
global O "nohead nolog"

global RS "tstat pval ci level(95)" 

quietly gsem ($R) ($T) ($M), $C $O
estimates store ALL
regsave using out/table2, replace addlabel(M, all) tstat pval ci level(95)

quietly gsem ($R) ($T) ($M) if G == "boy", $C $O
estimates store BOYS
regsave using out/table2, append addlabel(M, boys) tstat pval ci level(95)

quietly gsem ($R) ($T) ($M) if G == "girl", $C $O
estimates store GIRLS
regsave using out/table2, append addlabel(M, girls) tstat pval ci level(95)

*estimates table ALL BOYS GIRLS, star(.10 .05 .01) b(%8.2f)
*use out/table2, clear
*sort M

In [ ]:
%connect_info

In [ ]:
%%stata --data de

global R "r <- g@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global T "t <- b@k1 m@k2 n@k3 M1[id]@1 _cons@kk"
global M "g b <- m@k4 c@k5 M2[id]@1"
global C "covstruct(_lexogenous, diagonal) nocapslatent latent(M1 M2) cov(e.r*e.t) cov(e.g*e.b) cov(M1[id]*M2[id]) "
global O "nohead nolog"

global RS "tstat pval ci level(95)" 

gsem ($R) ($T) ($M), $C $O coeflegend



In [ ]:
dr = pandas.read_stata('out/table2.dta')
dr = dr.set_index(['M', 'var'])
dr = dr[['coef']]
dr = dr.T
dr.head()

In [ ]:
RA = dr['all']
RB = dr['boys']
RG = dr['girls']

RB.T

In [ ]:
a = RA['var(M1[id]):_cons']['coef']
b = RA['var(e.r):_cons']['coef']
c = RA['var(e.t):_cons']['coef']
p1 = a / (a+b)
p2 = a / (a+c)

In [ ]:
RA.T.head()

In [ ]:
de = RA['r:m']['coef']
ie = RA['g:m']['coef']*RA['r:m']['coef']
te = de + ie
print(de, ie, te)

In [ ]:
RG.xs('coef', 0).head(20)

In [ ]:
RB.xs('coef', 0).head(20)

The estimation results are summarized in Table 2. All level-1 fixed effects and level-2 random effects were found to be significant at the 0.01 level. To facilitate straightforward comparison of the causal effects, all variables were transformed/standardized to a mean of zero and standard deviation of one. Conclusions based on non-standardized values support identical inferences. 

The results are consistent with the relationships postulated by the functional model of bone development. Physical maturity and nutrition influence muscle strength, which in-turn, along osteoclast activity, and physical maturity influences bone properties. Although physical maturation has both direct and indirect impacts bone properties, physical maturation and muscle strength had similar total impact. In terms of standard unit changes, the direct impact of muscle strength and 
physical maturation on bone properties was {rg} and {rm}. However, physical maturation also influenced bone properties through its impact on muscle strength. The unit impact of physical maturation on grip strength was {gm}. Therefore. the indirect impact of physical maturation on bone properties was {irm}, resulting in a total impact of {trm}. 

Osteoclast activity had a similar effect in boys and girls ({bn} and {gn}). The impact of physical maturation on muscle strength was {DMM} points greater for boys than girls, albeit the difference is not significant. 


In [ ]:
Markdown(
"""

Table 2 summarizes the results from the multilevel models. 
To facilitate straightforward comparison of the causal effects, all variables were transformed/standardized to a mean of zero and standard deviation of one. Conclusions based on non-standardized values were similar and support identical inferences. 

"""
.format()
)

To facilitate straightforward comparison of the causal effects, all variables were transformed/standardized to a mean of zero and standard deviation of one. Conclusions based on non-standardized values were similar and support identical inferences. The results are also presented in Figure 2. The estimated fixed (slope) effects are presented along the straight-arrowed paths, and intercepts/constant terms are within the dependent variable terms. The estimated level-1 variances are beside circle terms, the level-2 variances are within the doubled-circled terms, and the covariances are along the doubled-headed curved-arrowed paths. All variables were significant at the 0.05 level. The model equations along with detailed results and supplemental analyses are available at https://github.com/kmongeon/exercise_physiology.

The constrained estimation results are consistent with the relationships postulated by the functional model of bone development. Specifically, physical maturity, nutrition, and behavioral modulators influence muscle strength, which in-turn, along osteoclast activity, and physical maturity and behavioral modulators influences bone properties  (p<0.05). Each standard deviation increase in physical maturation leads to increases in muscle strength by approximately a 0.60 standard deviations. In standard units, the impact of nutrition and physical activity on…. is 0.15 and 0.12, respectively. The bone properties equation coefficients are also interpreted as standard deviation unit changes. Note, the conditional fixed effects account for level-2 participant variances. A unit increase in osteoclast activity decreases bone SOS 0.16 units. Physical activity, physical maturation and muscle strength have positive returns. Each unit increases in physical activity increases bone SOS 0.05 units. The physical maturation and muscle strength effects are 0.31 and 0.33 standard units, respectively. Physical maturation, as well as the other muscle strength modulators, also an indirect influence on bone properties through muscle strength  (i.e., (∂bone_(i,j))/(∂〖muscle〗_(i,j) )  (∂muscle_(i,j))/(∂〖matu〗_(i,j) )=0.33×0.60=0.20). In total, the influence of physical maturation is 0.52 (= 0.33 + 0.20) standard units. 


In [ ]:
Markdown(
"""

a is {a:8.4f}

"""
.format
    (
a = RA['r:g']['coef']
    )
        )
